In [1]:
import pandas_datareader as pdr  # to install: !pip install pandas_datareader
from datetime import datetime
import os
import eda
import numpy as np
from outlier_report import outlier_report
import matplotlib.pyplot as plt
import seaborn as sns

## return of market index

In [2]:
# Here we extract the market information
market_index = ['^GSPC', '^DJI']
market_index_name = ['S&P500', 'DJ30']

start_time = datetime(2019, 3, 1)
end_time = datetime(2021, 4, 30)

# load
index_points = pdr.get_data_yahoo(market_index, start = start_time, end = end_time)
index_points = index_points.filter(like='Adj Close') # reduce to just columns with this in the name
index_points.columns = market_index_name # put their tickers as column names


# add return var.
index_points['sp500_rtn'] = index_points['S&P500'].pct_change()
index_points['DJ30_rtn'] = index_points['DJ30'].pct_change()


index_points.round(6).to_csv('input_data/market_index_return.csv')

## return of cryptos

In [3]:
# Here we extract 
cryptos = ['BTC-USD', 'ETH-USD', 'BNB-USD', 'XRP-USD', 'USDT-USD', 
           'DOGE-USD', 'ADA-USD', 'BCH-USD', 'LTC-USD', 'LINK-USD']

crypto_names = ['BTC', 'ETH', 'BNB', 'XRP', 'USDT', 
           'DOGE', 'ADA', 'BCH', 'LTC', 'LINK']

start_time = datetime(2019, 3, 1)
end_time = datetime(2021, 4, 30)

# load
crypto_prices = pdr.get_data_yahoo(cryptos, start = start_time, end = end_time)
crypto_prices = crypto_prices.filter(like='Adj Close') # reduce to just columns with this in the name
crypto_prices.columns = crypto_names # put their tickers as column names

# add return var.
# MAKE SURE YOU CREATE THE VARIABLES WITHIN EACH FIRM - use groupby
for ele in crypto_names:
    col_name = ele + '_rtn'
    crypto_prices[col_name] = crypto_prices[ele].pct_change()
    
crypto_prices.round(6).to_csv('input_data/cryptos_return.csv')

## volume of cryptos

In [4]:
# Here we extract 
cryptos2 = ['BTC-USD', 'ETH-USD', 'BNB-USD', 'XRP-USD', 'USDT-USD', 
           'DOGE-USD', 'ADA-USD', 'BCH-USD', 'LTC-USD', 'LINK-USD']

crypto_names2 = ['BTC', 'ETH', 'BNB', 'XRP', 'USDT', 
           'DOGE', 'ADA', 'BCH', 'LTC', 'LINK']

start_time = datetime(2019, 3, 1)
end_time = datetime(2021, 4, 30)

# load
crypto_prices2 = pdr.get_data_yahoo(cryptos2, start = start_time, end = end_time)
crypto_prices2 = crypto_prices2.filter(like='Volume') # reduce to just columns with this in the name
crypto_prices2.columns = crypto_names2 # put their tickers as column names
crypto_prices2
# add return var.
# MAKE SURE YOU CREATE THE VARIABLES WITHIN EACH FIRM - use groupby

    
crypto_prices2.round(4).to_csv('input_data/cryptos_volume.csv')

## crypto market return CCi

In [5]:
import pandas as pd

cci30 = pd.read_csv('input_data/cci30_OHLCV.csv')

cci30 = cci30.assign(ret = cci30['Close'].pct_change())

cci30.to_csv('input_data/cci30_OHLCV.csv',index=False)

## risk free rate

In [6]:
import pandas as pd
from requests_html import HTMLSession
session = HTMLSession()

frames = []

for y in ['2019','2020','2021']:
    url = 'https://www.treasury.gov/resource-center/data-chart-center/interest-rates/pages/TextView.aspx?data=yieldYear&year=' + y
    r = session.get(url)
    table = r.html.find('.t-chart')[0]
    df = pd.read_html(table.html)[0]
    df['Date'] = pd.to_datetime(df['Date'], format = '%m/%d/%y')
    if y == '2019':
        df = df.query('Date >= "03/01/19"')
    if y == '2021':
        df = df.query('Date <= "05/01/21"')
    frames.append(df)
    
risk_free_rate = pd.concat(frames)
risk_free_rate.to_csv('input_data/risk_free_rate.csv',index=False)